# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f71600cc9d0>
├── 'a' --> tensor([[ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113]])
└── 'x' --> <FastTreeValue 0x7f7160052340>
    └── 'c' --> tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                        [ 1.0403,  0.0853, -0.8829,  0.0701],
                        [-2.7398,  1.1966,  0.2083,  2.1392]])

In [4]:
t.a

tensor([[ 0.6887,  1.5104,  0.2301],
        [-1.1731,  0.3970,  0.9113]])

In [5]:
%timeit t.a

64.4 ns ± 0.0312 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f71600cc9d0>
├── 'a' --> tensor([[-0.3221, -1.6705, -0.1928],
│                   [-1.0414, -0.0711,  0.9267]])
└── 'x' --> <FastTreeValue 0x7f7160052340>
    └── 'c' --> tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                        [ 1.0403,  0.0853, -0.8829,  0.0701],
                        [-2.7398,  1.1966,  0.2083,  2.1392]])

In [7]:
%timeit t.a = new_value

71.8 ns ± 0.083 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113]]),
    x: Batch(
           c: tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                      [ 1.0403,  0.0853, -0.8829,  0.0701],
                      [-2.7398,  1.1966,  0.2083,  2.1392]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6887,  1.5104,  0.2301],
        [-1.1731,  0.3970,  0.9113]])

In [11]:
%timeit b.a

58.8 ns ± 0.0363 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7172, -0.5993,  0.5684],
               [ 2.7196, -0.2141,  1.3676]]),
    x: Batch(
           c: tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                      [ 1.0403,  0.0853, -0.8829,  0.0701],
                      [-2.7398,  1.1966,  0.2083,  2.1392]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.069 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

821 ns ± 0.809 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 7.47 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 804 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 468 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f70ab2e5850>
├── 'a' --> tensor([[[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]],
│           
│                   [[ 0.6887,  1.5104,  0.2301],
│                    [-1.1731,  0.3970,  0.9113]]])
└── 'x' --> <FastTreeValue 0x7f70ab2e5a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f70ab2e5880>
├── 'a' --> tensor([[ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113],
│                   [ 0.6887,  1.5104,  0.2301],
│                   [-1.1731,  0.3970,  0.9113]])
└── 'x' --> <FastTreeValue 0x7f70ab2e5610>
    └── 'c' --> tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                        [ 1.0403,  0.0853, -0.8829,  0.0701],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 52.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.8 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]],
       
               [[ 0.6887,  1.5104,  0.2301],
                [-1.1731,  0.3970,  0.9113]]]),
    x: Batch(
           c: tensor([[[ 0.4959, -0.5341,  2.2592,  0.8148],
                       [ 1.0403,  0.0853, -0.8829,  0.0701],
                       [-2.7398,  1.1966,  0.2083,  2.1392]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113],
               [ 0.6887,  1.5104,  0.2301],
               [-1.1731,  0.3970,  0.9113]]),
    x: Batch(
           c: tensor([[ 0.4959, -0.5341,  2.2592,  0.8148],
                      [ 1.0403,  0.0853, -0.8829,  0.0701],
                      [-2.7398,  1.1966,  0.2083,  2.1392],
                      [ 0.4959, -0.5341,  2.2592,  0.8148],
                      [ 1.0403,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
